In [4]:
import os

def parse_block_file(file_path):
    """
    Parse the file to extract a mapping of participant runs to blocks.
    """
    with open(file_path, 'r') as file:
        lines = file.readlines()

    # Dictionary to hold the mapping
    mapping = {}

    participant_id = None
    for line in lines:
        line = line.strip()
        if not line:
            continue
        if line.startswith("SF"):
            participant_id = line
            mapping[participant_id] = {}
        elif line.startswith("Run"):
            run_parts = line.split(":")
            run_id = run_parts[0].strip()
            blocks = run_parts[1].strip().split(',')
            mapping[participant_id][run_id] = [block.strip() for block in blocks]

    return mapping

# Get the mapping from the file
run_to_block_mapping = parse_block_file("Z:\Lab\StabFlex\Data\WhatBlocksAreWhichRuns.txt")
run_to_block_mapping

{'SF01': {'Run 1': ['B', 'A'],
  'Run 2': ['D', 'C'],
  'Run 3': ['C', 'D'],
  'Run 4': ['B', 'A']},
 'SF02': {'Run 1': ['A', 'B'],
  'Run 2': ['D', 'C'],
  'Run 3': ['D', 'A'],
  'Run 4': ['C', 'B']},
 'SF03': {'Run 1': ['B', 'C'],
  'Run 2': ['A', 'D'],
  'Run 3': ['C', 'D'],
  'Run 4': ['B', 'A']},
 'SF04': {'Run 1': ['B', 'C'],
  'Run 2': ['A', 'D'],
  'Run 3': ['D', 'A'],
  'Run 4': ['C', 'B']},
 'SF05': {'Run 1': ['C', 'D'],
  'Run 2': ['B', 'A'],
  'Run 3': ['D', 'A'],
  'Run 4': ['C', 'B']},
 'SF06': {'Run 1': ['A', 'B'],
  'Run 2': ['D', 'C'],
  'Run 3': ['B', 'C'],
  'Run 4': ['A', 'D']},
 'SF07': {'Run 1': ['A', 'B'],
  'Run 2': ['D', 'C'],
  'Run 3': ['C', 'D'],
  'Run 4': ['B', 'A']},
 'SF08': {'Run 1': ['A', 'B'],
  'Run 2': ['D', 'C'],
  'Run 3': ['D', 'A'],
  'Run 4': ['C', 'B']},
 'SF09': {'Run 1': ['B', 'C'],
  'Run 2': ['A', 'D'],
  'Run 3': ['C', 'D'],
  'Run 4': ['B', 'A']},
 'SF10': {'Run 1': ['B', 'C'],
  'Run 2': ['A', 'D'],
  'Run 3': ['D', 'A'],
  'Run 4': ['C

In [5]:
def get_block_for_run(participant_id, run_id):
    """
    Get the block for a given run using the extracted mapping.
    """
    # Extract the block from the mapping
    blocks_for_run = run_to_block_mapping.get(participant_id, {}).get(run_id, [])

    # For the current implementation, we'll assume that only the first block is relevant for the EV.
    # If this is not the case, modifications can be made.
    return blocks_for_run[0] if blocks_for_run else None

# Test the function
test_participant_id = "SF02"
test_run_id = "Run 1"
get_block_for_run(test_participant_id, test_run_id)

'A'

In [38]:
def get_ev_for_input(input_path):
    """
    Determine the EV for a given input path.
    """
    # Extract relevant identifiers from the input path
    base_name = os.path.basename(input_path)
    parts = os.path.dirname(input_path).split('/')
    
    # Extract participant_id and run_id based on the updated rules
    partial_id = parts[-2].split('_')
    participant_id = partial_id[0]
    run_id = "Run " + partial_id[2][0]
    
    cope_num = int(base_name.split('.')[0][4:])  # Extracting the number from copeX
    
    # Determine the block for this run
    block = get_block_for_run(participant_id, run_id)
    if not block:
        return None  # Return None if block info is not found

    # Determine the congruency and switch/repeat based on the cope number
    cope_to_info = {
        1: ("i", "s", 0),
        2: ("i", "s", 1),
        3: ("i", "r", 0),
        4: ("i", "r", 1),
        5: ("c", "s", 0),
        6: ("c", "s", 1),
        7: ("c", "r", 0),
        8: ("c", "r", 1),
    }
    
    congruency, switch_or_repeat, block_idx = cope_to_info[cope_num]

    # Map block letter to congruency and switch proportions
    block_to_info = {
        "A": ("25", "25"),
        "B": ("25", "75"),
        "C": ("75", "25"),
        "D": ("75", "75"),
    }
    
    # Ensure the block value is valid before accessing the dictionary
    congruency_prop, switch_prop = block_to_info.get(block, (None, None))
    if not congruency_prop or not switch_prop:
        return None

    # Construct the EV identifier using the extracted information
    # Note: For block 2 (block_idx = 1), we use the opposite block's info
    if block_idx == 1:
        opposite_block = {"A": "B", "B": "A", "C": "D", "D": "C"}
        congruency_prop, switch_prop = block_to_info[opposite_block[block]]

    ev = f"{congruency}{congruency_prop}{switch_or_repeat}{switch_prop}"

    return ev

# Testing the updated extraction logic
test_input_path = "/mnt/munin2/Egner/Lab/StabFlex/Analysis/Univariate/SF02/SF02_RUN_1/SF02_RUN_1.feat/stats/cope8.nii.gz"
ev_for_test_input = get_ev_for_input(test_input_path)
ev_for_test_input


'c25r75'

In [39]:
get_ev_for_input("/mnt/munin2/Egner/Lab/StabFlex/Analysis/Univariate/SF02/SF02_RUN_1/SF02_RUN_1.feat/stats/cope1.nii.gz")

'i25s25'

In [41]:
def generate_ev_matrix(input_paths):
    """
    Generate the EV matrix for a list of input paths.
    """
    # Define all possible EVs
    evs = ["i25s25", "i25s75", "i75s25", "i75s75", "i25r25", "i25r75", "i75r25", "i75r75", "c25s25", "c25s75", "c75s25", "c75s75", "c25r25", "c25r75", "c75r25", "c75r75"]

    # Initialize matrix
    matrix = []

    for input_path in input_paths:
        ev_for_input = get_ev_for_input(input_path)
        row = [1 if ev == ev_for_input else 0 for ev in evs]
        matrix.append(row)

    return matrix

# Example usage (using a single path for demonstration):
input_paths = ["/mnt/munin2/Egner/Lab/StabFlex/Analysis/Univariate/SF02/SF02_RUN_1/SF02_RUN_1.feat/stats/cope1.nii.gz", "/mnt/munin2/Egner/Lab/StabFlex/Analysis/Univariate/SF02/SF02_RUN_1/SF02_RUN_1.feat/stats/cope2.nii.gz"]
ev_matrix = generate_ev_matrix(input_paths)
ev_matrix


[[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]